In [1]:
import os
import time
import torch
from torch import nn, optim
from torch import tensor
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt
from PIL import Image
from sklearn.metrics import r2_score, precision_score, recall_score, f1_score, confusion_matrix

In [2]:
class FaceDataset(Dataset):
    def __init__(self, index_file):
        self.df = pd.read_csv(open(index_file))
    
    def __len__(self):
        return self.df.shape[0]
    
    def __getitem__(self, i):
        Y = tensor(self.df.iloc[i][0])
        X = tensor(self.df.iloc[i][1:])
        return (X, Y)

In [3]:
train_data = FaceDataset('/home/cse/dual/cs5180404/col774/train.csv')
batch_size = 32
train_data_loader = DataLoader(train_data, batch_size, shuffle=True, num_workers=4)

In [4]:
class NNet(nn.Module):
    def __init__(self):
        super(NNet, self).__init__()
        self.fc1 = nn.Linear(2304, 100)
        self.fc2 = nn.Linear(100, 7)
        self.output = nn.Softmax(dim=1)
    
    def forward(self, x):
        x = x.view(-1, 2304)
        f1 = F.relu(self.fc1(x))
        f2 = F.relu(self.fc2(f1))
        return self.output(f2)

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
torch.cuda.set_device(0)

net = NNet()
net = net.to(device)

loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=1e-4)
print(net)

cuda
NNet(
  (fc1): Linear(in_features=2304, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=7, bias=True)
  (output): Softmax()
)


In [6]:
total_loss = 0
for epoch in range(10):
    for i, data in enumerate(train_data_loader, 0):
        images, classes = data
        images, classes = images.to(device).float(), classes.to(device).long()
        #images, classes = images.float(), classes.long()
        optimizer.zero_grad()
        output = net(images)
        #print(output)
        
        loss = loss_function(output, classes)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item() 
        if(i % 20 == 19):
            print("After epoch {}, mini-batch {}, loss: {}".format(epoch + 1, i + 1, total_loss))
            correct = 0
            total = 0
            total_loss = 0
    total_loss = 0

After epoch 1, mini-batch 20, loss: 38.88921129703522
After epoch 1, mini-batch 40, loss: 38.5589919090271
After epoch 1, mini-batch 60, loss: 38.37402963638306
After epoch 1, mini-batch 80, loss: 37.569982051849365
After epoch 1, mini-batch 100, loss: 37.74152374267578
After epoch 1, mini-batch 120, loss: 37.39133965969086
After epoch 1, mini-batch 140, loss: 37.618449687957764
After epoch 1, mini-batch 160, loss: 37.07066833972931
After epoch 1, mini-batch 180, loss: 37.17407929897308
After epoch 1, mini-batch 200, loss: 37.277369260787964
After epoch 1, mini-batch 220, loss: 36.87316644191742
After epoch 1, mini-batch 240, loss: 36.95002233982086
After epoch 1, mini-batch 260, loss: 37.07855558395386
After epoch 1, mini-batch 280, loss: 36.98200750350952
After epoch 1, mini-batch 300, loss: 36.686439514160156
After epoch 1, mini-batch 320, loss: 36.987884759902954
After epoch 1, mini-batch 340, loss: 36.60460722446442
After epoch 1, mini-batch 360, loss: 36.73640716075897
After epoc

After epoch 5, mini-batch 600, loss: 35.48583102226257
After epoch 6, mini-batch 20, loss: 34.415576100349426
After epoch 6, mini-batch 40, loss: 34.540515422821045
After epoch 6, mini-batch 60, loss: 34.48539972305298
After epoch 6, mini-batch 80, loss: 34.61926865577698
After epoch 6, mini-batch 100, loss: 34.95616018772125
After epoch 6, mini-batch 120, loss: 34.597689628601074
After epoch 6, mini-batch 140, loss: 35.46968472003937
After epoch 6, mini-batch 160, loss: 34.529619455337524
After epoch 6, mini-batch 180, loss: 35.36648452281952
After epoch 6, mini-batch 200, loss: 34.32918167114258
After epoch 6, mini-batch 220, loss: 35.14157032966614
After epoch 6, mini-batch 240, loss: 34.77009344100952
After epoch 6, mini-batch 260, loss: 34.483880400657654
After epoch 6, mini-batch 280, loss: 35.180511474609375
After epoch 6, mini-batch 300, loss: 35.025996804237366
After epoch 6, mini-batch 320, loss: 34.78375005722046
After epoch 6, mini-batch 340, loss: 34.34204924106598
After e

After epoch 10, mini-batch 580, loss: 33.56989228725433
After epoch 10, mini-batch 600, loss: 33.86881935596466


In [55]:
train_data = FaceDataset('/home/cse/dual/cs5180404/col774/public_test.csv')
batch_size = 32
test_data_loader = DataLoader(train_data, batch_size, shuffle=False, num_workers=4)

In [65]:
actual, prediction = [], []
for i, data in enumerate(test_data_loader, 0):
    images, classes = data
    #images, classes = images.to(device).float(), classes.to(device).long()
    images, classes = images.float(), classes.long()
    actual.extend(classes.squeeze().tolist())
    
    optimizer.zero_grad()
    output = net(images)
    
    prediction.extend(torch.argmax(output, dim=1).squeeze().tolist())
    #print(output)
print(f1_score(actual, prediction, average='macro'))
print(confusion_matrix(actual, prediction))

0.3162514714552039
[[ 145    0   92  301  162   45  147]
 [  11    0   10   38    8    8   23]
 [  73    0  153  255  184   96  158]
 [  38    0   80 1145  171   50  137]
 [  99    0   89  283  345   47  234]
 [  25    0   61  135   56  359   81]
 [  67    0   78  291  157   58  463]]


In [67]:
print(np.bincount(np.array(actual)))
print(np.bincount(np.array(prediction)))

[ 892   98  919 1621 1097  717 1114]
[ 458    0  563 2448 1083  663 1243]
